# LAB | Ensemble Methods

**Load the data**

In this challenge, we will be working with the same Spaceship Titanic data, like the previous Lab. The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

In this Lab, you should try different ensemble methods in order to see if can obtain a better model than before. In order to do a fair comparison, you should perform the same feature scaling, engineering applied in previous Lab.

In [153]:
from sklearn.datasets import  fetch_california_housing
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor, BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [132]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [133]:
spaceship.dropna(inplace=True)

Now perform the same as before:
- Feature Scaling
- Feature Selection


In [134]:
#spliting cabin columns into 3 separate ones
spaceship[['Deck', 'CabinNum', 'Side']] = spaceship['Cabin'].str.split('/', expand=True)
spaceship.drop(columns=['PassengerId', 'Name', 'Cabin'], inplace= True)

In [135]:
spaceship['CabinNum'] = spaceship['CabinNum'].astype(int)

In [136]:
#getting dummies to convert catogerical columns
spaceship = pd.get_dummies(spaceship, drop_first=True)
spaceship

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,CabinNum,HomePlanet_Europa,HomePlanet_Mars,...,Destination_TRAPPIST-1e,VIP_True,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Side_S
0,39.0,0.0,0.0,0.0,0.0,0.0,False,0,True,False,...,True,False,True,False,False,False,False,False,False,False
1,24.0,109.0,9.0,25.0,549.0,44.0,True,0,False,False,...,True,False,False,False,False,False,True,False,False,True
2,58.0,43.0,3576.0,0.0,6715.0,49.0,False,0,True,False,...,True,True,False,False,False,False,False,False,False,True
3,33.0,0.0,1283.0,371.0,3329.0,193.0,False,0,True,False,...,True,False,False,False,False,False,False,False,False,True
4,16.0,303.0,70.0,151.0,565.0,2.0,True,1,False,False,...,True,False,False,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,41.0,0.0,6819.0,0.0,1643.0,74.0,False,98,True,False,...,False,True,False,False,False,False,False,False,False,False
8689,18.0,0.0,0.0,0.0,0.0,0.0,False,1499,False,False,...,False,False,False,False,False,False,False,True,False,True
8690,26.0,0.0,0.0,1872.0,1.0,0.0,True,1500,False,False,...,True,False,False,False,False,False,False,True,False,True
8691,32.0,0.0,1049.0,0.0,353.0,3235.0,False,608,True,False,...,False,False,False,False,False,True,False,False,False,True


In [137]:
spaceship['Transported'] = spaceship['Transported'].astype(int)

**Perform Train Test Split**

In [138]:
features = spaceship.drop(columns='Transported')
target = spaceship['Transported']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=0)

**Model Selection** - now you will try to apply different ensemble methods in order to get a better model

- Bagging and Pasting

In [139]:
bagging_reg = BaggingClassifier(DecisionTreeClassifier(max_depth=20),
                               n_estimators=1200,
                               max_samples = 5000,
                               bootstrap=False)

In [140]:
bagging_reg.fit(X_train, y_train)

BaggingClassifier(bootstrap=False,
                  estimator=DecisionTreeClassifier(max_depth=20),
                  max_samples=5000, n_estimators=1200)

In [141]:
pred = bagging_reg.predict(X_test)

print("Accuracy:", accuracy_score(y_test, pred))
print("\nClassification Report:\n", classification_report(y_test, pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred))

Accuracy: 0.7783661119515886

Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.79      0.78       661
           1       0.78      0.77      0.78       661

    accuracy                           0.78      1322
   macro avg       0.78      0.78      0.78      1322
weighted avg       0.78      0.78      0.78      1322


Confusion Matrix:
 [[520 141]
 [152 509]]


- Random Forests

In [142]:
forest = RandomForestClassifier(n_estimators=200, max_depth=10)

In [143]:
forest.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, n_estimators=200)

In [144]:
pred = forest.predict(X_test)

print("Accuracy:", accuracy_score(y_test, pred))
print("\nClassification Report:\n", classification_report(y_test, pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred))

Accuracy: 0.7934947049924357

Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.79      0.79       661
           1       0.79      0.79      0.79       661

    accuracy                           0.79      1322
   macro avg       0.79      0.79      0.79      1322
weighted avg       0.79      0.79      0.79      1322


Confusion Matrix:
 [[524 137]
 [136 525]]


- Gradient Boosting

In [150]:
gb_clas = GradientBoostingClassifier(max_depth=12, n_estimators=30)

In [151]:
gb_clas.fit(X_train, y_train)

GradientBoostingClassifier(max_depth=12, n_estimators=30)

In [152]:
pred = gb_clas.predict(X_test)

print("Accuracy:", accuracy_score(y_test, pred))
print("\nClassification Report:\n", classification_report(y_test, pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred))

Accuracy: 0.7866868381240545

Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.81      0.79       661
           1       0.80      0.77      0.78       661

    accuracy                           0.79      1322
   macro avg       0.79      0.79      0.79      1322
weighted avg       0.79      0.79      0.79      1322


Confusion Matrix:
 [[534 127]
 [155 506]]


- Adaptive Boosting

In [154]:
ada_clas = AdaBoostClassifier(DecisionTreeClassifier(max_depth=12),
                            n_estimators=300)

ada_clas.fit(X_train, y_train)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=12),
                   n_estimators=300)

In [155]:
pred = ada_clas.predict(X_test)
print("Accuracy:", accuracy_score(y_test, pred))
print("\nClassification Report:\n", classification_report(y_test, pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, pred))

Accuracy: 0.7700453857791225

Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.78      0.77       661
           1       0.77      0.76      0.77       661

    accuracy                           0.77      1322
   macro avg       0.77      0.77      0.77      1322
weighted avg       0.77      0.77      0.77      1322


Confusion Matrix:
 [[514 147]
 [157 504]]


Which model is the best and why?

In [ ]:
# Random Forest was the best model because it achieved the highest accuracy (around 79.1%) and the best F1-score (0.79) for both classes